In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "2"  # Set the GPU 2 to use

## 무비렌즈

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
# gpu 설정
use_cuda = True

use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

### 데이터 준비

In [3]:
# 데이터 로드

"""
필요한 컬럼은 유저, 아이템, rating

우선 간단하게 빨리 해보는게 중요하니,
rating이 5점이면 rating 컬럼을 1
아니라면 0로 바꾸자고.
"""


def getDataByScenario(scenario):
    """
    :param scenario: increase, fixed, user, item
    :return: dfs
    """
    dfs = []

    for i in range(6):
        df = pd.read_csv(f"./dataset/Movielens/{scenario}/ml_1m_inc{i}.csv")
        dfs.append(df)
    return dfs

In [4]:
len(getDataByScenario("item"))

6

## dataloader 정의

In [5]:
class MovielensDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.iloc[idx]['user']
        item = self.df.iloc[idx]['item']
        rating = self.df.iloc[idx]['rating']
        return user, item, rating

## 모델 정의

In [6]:
# NCF 모델
class NCF(nn.Module):
    def __init__(self, n_users, n_movies, emb_size=8, hidden_size=64):
        super(NCF, self).__init__()
        self.user_embedding = nn.Embedding(n_users, emb_size)
        self.movie_embedding = nn.Embedding(n_movies, emb_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(emb_size * 2, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, user_input, movie_input):
        user_embedded = self.user_embedding(user_input)
        movie_embedded = self.movie_embedding(movie_input)
        input_concat = torch.cat([user_embedded, movie_embedded], dim=-1)
        prediction = self.fc_layers(input_concat)
        return prediction

## 모델 train/test 함수 정의

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [8]:
def recall_at_k(output, target, k):
    if len(output) < k:
        k = len(output)
    _, idx = torch.topk(output, k=k)
    hit = torch.sum(target[idx])
    return hit.float() / target.sum().float() if target.sum().float() else torch.Tensor([0])


def test(model, device, test_loader, k=20):
    model.eval()
    criterion = nn.BCELoss()

    test_loss = 0
    test_recall = 0
    with torch.no_grad():
        for user, item, rating in test_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            output = model(user, item).squeeze()
            loss = criterion(output, rating.float())
            test_loss += loss.item()
            test_recall += recall_at_k(output, rating, k).item()  # recall@20 기준
    test_loss /= len(test_loader)
    test_recall /= len(test_loader)

    return test_loss, test_recall

### 모델 학습

1. Full
2. EWC

In [9]:
### Config..
EPOCH = 1
SEED = 42
BATCH_SIZE = 64
N_USER = 6040
N_ITEM = 3952

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

1. Full

In [10]:
def concat_df(dfs, k):
    return pd.concat(dfs[:k+1], axis=0)

In [13]:
def getFullResultByScenario(scenario):
    dfs = getDataByScenario(scenario)

    inference_recall_list = []
    for i in range(len(dfs)-1):

        # 모델 객체 생성
        n_users = N_USER + 1
        n_movies = N_ITEM + 1
        model = NCF(n_users, n_movies).to(device)
        # 옵티마이저 설정
        optimizer = optim.Adam(model.parameters(), lr=0.001)


        # train
        train_dataset = MovielensDataset(concat_df(dfs, i))
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        for e in tqdm(range(1, EPOCH+1)):
            train(model, device, train_loader, optimizer, e)

        # inference test
        inference_dataset = MovielensDataset(dfs[i+1])
        inference_loader = DataLoader(inference_dataset, batch_size=BATCH_SIZE, shuffle=True)

        _, inference_recall = test(model, device, inference_loader)
        inference_recall_list.append(inference_recall)
        print(f"******* {scenario} scenario At TASK{i} inference recall20 = {inference_recall}\n")

        # forgetting test
        # i == 0 인 경우는 제외
        if i == 0 :
            continue
        forgetting_dataset_list = dfs[:i]

        forgetting_recall_list = []
        for j, f in enumerate(forgetting_dataset_list):
            forgetting_dataset = MovielensDataset(f)
            forgetting_loader = DataLoader(forgetting_dataset, batch_size=BATCH_SIZE, shuffle=True)
            _, forgetting_recall = test(model, device, forgetting_loader)
            forgetting_recall_list.append(forgetting_recall)
            print(f"******* {scenario} scenario At TASK{i} prev TASK{j} forgetting recall20 = {forgetting_recall}")
        average_forgetting_recall = sum(forgetting_recall_list) / len(forgetting_recall_list)
        print(f"\n******* {scenario} scenario At TASK{i} forgetting recall20 average = {average_forgetting_recall}")

    average_inference_recall = sum(inference_recall_list)/len(inference_recall_list)
    print(f"******* {scenario} scenario At TASK{i} inference recall20 average = {average_inference_recall}")

    return average_inference_recall, average_forgetting_recall

In [14]:
full_increase_inference, full_increase_forgetting  = getFullResultByScenario("increase")

100%|██████████| 1/1 [01:31<00:00, 91.17s/it]


******* increase scenario At TASK0 inference recall20 = 0.5192720418731817



100%|██████████| 1/1 [01:44<00:00, 104.14s/it]


******* increase scenario At TASK1 inference recall20 = 0.5413823056998875

******* increase scenario At TASK0 forgetting recall20 = 0.6248394272245803

******* increase scenario At TASK0 forgetting recall20 average = 0.6248394272245803


100%|██████████| 1/1 [01:55<00:00, 115.82s/it]


******* increase scenario At TASK2 inference recall20 = 0.5617415812179697

******* increase scenario At TASK0 forgetting recall20 = 0.6273606803631889
******* increase scenario At TASK1 forgetting recall20 = 0.6325205555073827

******* increase scenario At TASK1 forgetting recall20 average = 0.6299406179352858


100%|██████████| 1/1 [02:09<00:00, 129.31s/it]


******* increase scenario At TASK3 inference recall20 = 0.5529543891596832

******* increase scenario At TASK0 forgetting recall20 = 0.6327924912771947
******* increase scenario At TASK1 forgetting recall20 = 0.6354449129491162
******* increase scenario At TASK2 forgetting recall20 = 0.665458995428345

******* increase scenario At TASK2 forgetting recall20 average = 0.644565466551552


100%|██████████| 1/1 [02:21<00:00, 141.59s/it]


******* increase scenario At TASK4 inference recall20 = 0.6346755686691038

******* increase scenario At TASK0 forgetting recall20 = 0.6318242966864538
******* increase scenario At TASK1 forgetting recall20 = 0.6342657711670053
******* increase scenario At TASK2 forgetting recall20 = 0.6737529997161333
******* increase scenario At TASK3 forgetting recall20 = 0.6512151541854975

******* increase scenario At TASK3 forgetting recall20 average = 0.6477645554387724


In [ ]:
full_fixed_inference, full_fixed_forgetting = getFullResultByScenario("fixed")

In [ ]:
full_user_inference, full_user_forgetting = getFullResultByScenario("user")

In [ ]:
full_item_inference, full_item_forgetting = getFullResultByScenario("item")

2. EWC

In [17]:
# Task가 끝날 때 마다 optpar와 fisher를 저장해주는 함수.
def on_task_update(model, device, train_loader, optimizer, task_id, fisher_dict, optpar_dict):
    model.train()
    criterion = nn.BCELoss()
    optimizer.zero_grad()

    # accumulating gradients
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        loss.backward()

    fisher_dict[task_id] = {}
    optpar_dict[task_id] = {}

    # gradients accumulated can be used to calculate fisher
    for name, param in model.named_parameters():
        fisher_dict[task_id][name] = param.grad.data.clone().pow(2)  # 누적 grad 값
        optpar_dict[task_id][name] = param.data.clone()  # 최적 grad 값

In [18]:
# EWC를 적용한 train 함수
def train_ewc(model, device, train_loader, optimizer, epoch, task_id, fisher_dict, optpar_dict, ewc_lambda):
    model.train()
    criterion = nn.BCELoss()

    train_loss = 0
    for user, item, rating in train_loader:
        user, item, rating = user.to(device), item.to(device), rating.to(device)
        optimizer.zero_grad()
        output = model(user, item).squeeze()
        loss = criterion(output, rating.float())
        train_loss += loss.item()

        # EWC 적용 부분
        for task in range(task_id):
            for name, param in model.named_parameters():
                fisher = fisher_dict[task][name]
                optpar = optpar_dict[task][name]
                train_loss += (fisher * (optpar - param).pow(2)).sum() * ewc_lambda

        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    # print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, train_loss))

    return train_loss

In [19]:
def getEWCResultByScenario(scenario):
    dfs = getDataByScenario(scenario)

    # 모델 객체 생성
    n_users = N_USER + 1
    n_movies = N_ITEM + 1
    model = NCF(n_users, n_movies).to(device)
    # 옵티마이저 설정
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # EWC에 필요한 변수
    fisher_dict = {}
    optpar_dict = {}
    ewc_lambda = 0.4  # ewc 강도 조절.. 높을수록 이전 파라미터의 중요도가 높아짐

    inference_recall_list = []
    for i in range(len(dfs)-1):
        # train
        train_dataset = MovielensDataset(dfs[i])
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

        for e in tqdm(range(1, EPOCH+1)):
            train_ewc(model, device, train_loader, optimizer, e, i, fisher_dict, optpar_dict, ewc_lambda)
        on_task_update(model, device, train_loader, optimizer, i, fisher_dict, optpar_dict)

        # inference test
        inference_dataset = MovielensDataset(dfs[i+1])
        inference_loader = DataLoader(inference_dataset, batch_size=BATCH_SIZE, shuffle=True)

        _, inference_recall = test(model, device, inference_loader)
        inference_recall_list.append(inference_recall)
        print(f"******* {scenario} scenario At TASK{i} inference recall20 = {inference_recall}\n")

        # forgetting test
        # i == 0 인 경우는 제외
        if i == 0 :
            continue
        forgetting_dataset_list = dfs[:i]

        forgetting_recall_list = []
        for j, f in enumerate(forgetting_dataset_list):
            forgetting_dataset = MovielensDataset(f)
            forgetting_loader = DataLoader(forgetting_dataset, batch_size=BATCH_SIZE, shuffle=True)
            _, forgetting_recall = test(model, device, forgetting_loader)
            forgetting_recall_list.append(forgetting_recall)
            print(f"******* {scenario} scenario At TASK{i} prev TASK {j} forgetting recall20 = {forgetting_recall}")
        average_forgetting_recall = sum(forgetting_recall_list) / len(forgetting_recall_list)
        print(f"\n******* {scenario} scenario At TASK{i} forgetting recall20 average = {average_forgetting_recall}")

    average_inference_recall = sum(inference_recall_list)/len(inference_recall_list)
    print(f"******* {scenario} scenario At TASK{i} inference recall20 average = {average_inference_recall}")

    return average_inference_recall, average_forgetting_recall

In [ ]:
ewc_increase_inference, ewc_increase_forgetting = getEWCResultByScenario("increase")

100%|██████████| 1/1 [01:24<00:00, 84.64s/it]


******* increase scenario At TASK0 inference recall20 = 0.5323513580070103



100%|██████████| 1/1 [00:12<00:00, 12.20s/it]


******* increase scenario At TASK1 inference recall20 = 0.5622014968370418

******* increase scenario At TASK1 prev TASK 0 forgetting recall20 = 0.6055163651604514

******* increase scenario At TASK1 forgetting recall20 average = 0.6055163651604514


100%|██████████| 1/1 [00:12<00:00, 12.79s/it]


******* increase scenario At TASK2 inference recall20 = 0.5577268948237165

******* increase scenario At TASK2 prev TASK 0 forgetting recall20 = 0.597367920396904
******* increase scenario At TASK2 prev TASK 1 forgetting recall20 = 0.6134630786694938

******* increase scenario At TASK2 forgetting recall20 average = 0.6054154995331988


100%|██████████| 1/1 [00:14<00:00, 14.87s/it]


******* increase scenario At TASK3 inference recall20 = 0.5459518901604095

******* increase scenario At TASK3 prev TASK 0 forgetting recall20 = 0.6069247370581422
******* increase scenario At TASK3 prev TASK 1 forgetting recall20 = 0.6151531842038381
******* increase scenario At TASK3 prev TASK 2 forgetting recall20 = 0.6588088324229368

******* increase scenario At TASK3 forgetting recall20 average = 0.6269622512283056


100%|██████████| 1/1 [00:15<00:00, 15.14s/it]


In [ ]:
ewc_fixed_inference, ewc_fixed_forgetting = getEWCResultByScenario("fixed")

In [ ]:
ewc_user_inference, ewc_user_forgetting = getEWCResultByScenario("user")

In [ ]:
ewc_item_inference, ewc_item_forgetting = getEWCResultByScenario("item")

## 결과 확인

In [ ]:
print(f"""
full_increase_inference: {full_increase_inference} full_increase_inference: {full_increase_forgetting}
full_fixed_inference: {full_fixed_inference} full_increase_inference: {full_fixed_forgetting}
full_user_inference: {full_user_inference} full_increase_inference: {full_user_forgetting}
full_item_inference: {full_item_inference} full_increase_inference: {full_item_forgetting}
""")

In [ ]:
print(f"""
ewc_increase_inference: {ewc_increase_inference} ewc_increase_inference: {ewc_increase_forgetting}
ewc_fixed_inference: {ewc_fixed_inference} ewc_increase_inference: {ewc_fixed_forgetting}
ewc_user_inference: {ewc_user_inference} ewc_increase_inference: {ewc_user_forgetting}
ewc_item_inference: {ewc_item_inference} ewc_increase_inference: {ewc_item_forgetting}
""")